In [ ]:
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from time import perf_counter
import os
from os.path import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
contentPath=os.getcwd()
genPath=join(contentPath, 'DataGenerator')

import tflite_model_maker
from tflite_model_maker import image_classifier, model_spec
from tflite_model_maker.image_classifier import DataLoader
print(tflite_model_maker.__version__)

In [ ]:
# Load input data specific to an on-device ML app.
data = DataLoader.from_folder(genPath)
train_data, restData = data.split(0.9)
print(train_data.size//32)
validation_data, test_data = restData.split(0.5)

In [ ]:
# help(image_classifier.create)

In [ ]:
start=perf_counter()
# Customize the TensorFlow model.
model = image_classifier.create(
    train_data,
    model_spec=model_spec.get('mobilenet_v2'),
    epochs=1,
    dropout_rate=0.25,
    learning_rate=0.001,
    momentum=0.9)

finish=perf_counter()
cvu.printTime(start, finish)

In [ ]:
# accuracy: 90.0%
# Evaluate the model.
loss, accuracy = model.evaluate(test_data)

loss = round(loss, 4)
accuracy = round(accuracy, 1)
print(f'loss: {loss}\taccuracy: {accuracy * 100}%')

q

In [ ]:
# Export to Tensorflow Lite model and label file in `export_dir`.
# model.export(export_dir='/tmp/')
model.export(export_dir=contentPath)